# Readme
This is a demo notebook that shows how the core schema is loaded and how to add data

If you have not installed `djimaging`, please go to project directory and install it via `pip install --editable .`.

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Base

## Prepare dj config

In [ ]:
import os
import datajoint as dj

config_file = '../djimaging/djconfig/djconf_template.json'  # WARNING: This will not work with the empty template

assert os.path.isfile(config_file), 'File not found'

# Load configuration for user
dj.config.load(config_file)

## Create or load schema

In [ ]:
from djimaging.schemas.core_schema import *

In [ ]:
schema

Specify your schema name here.

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, schema_name="your_schema_name", create_schema=True, create_tables=True)

In [ ]:
schema

In [ ]:
dj.ERD(schema)

## Upload user

In [ ]:
# Enter user last name
experimenter = input()

In [ ]:
# Enter the full path to your data
data_dir = input()

In [ ]:
userinfo = {
    'experimenter': experimenter, 
    'data_dir': data_dir,
    'pre_data_dir': 'Pre',
    'raw_data_dir': 'Raw',
    'datatype_loc': 0,
    'animal_loc': 1,
    'region_loc': 2,
    'field_loc': 3,
    'stimulus_loc': 4,
    'condition_loc': 5,
}

assert os.path.isdir(userinfo['data_dir'])

In [ ]:
UserInfo().upload_user(userinfo)
UserInfo()

## Populate data

### Experiments

In [ ]:
# Before scan
Experiment()

In [ ]:
Experiment().rescan_filesystem(verbose=True)

In [ ]:
# After scan
Experiment()

### Fields

In [ ]:
# Before scan
Field()

In [ ]:
Field().rescan_filesystem(verbose=2)

In [ ]:
# After scan
Field()

In [ ]:
Field().plot1(Field().fetch(as_dict=True)[0])

#### Rois

In [ ]:
Roi().populate()

In [ ]:
Roi()

In [ ]:
Roi().plot1(Roi().fetch(as_dict=True)[0])

### Stimuli

#### Add default stimuli

In [ ]:
Stimulus()

In [ ]:
Stimulus().add_nostim(skip_duplicates=True)
Stimulus().add_chirp(spatialextent=1000, stim_v='global', key=dict(stim_name='gChirp', alias="chirp_gchirp_globalchirp"), skip_duplicates=True)
Stimulus().add_chirp(spatialextent=300, stim_v='local', key=dict(stim_name='lChirp', alias="lchirp_localchirp"), skip_duplicates=True)
Stimulus().add_noise(pix_n_x=20, pix_n_y=15, pix_scale_x_um=30, pix_scale_y_um=30, skip_duplicates=True)
Stimulus().add_movingbar(skip_duplicates=True)

In [ ]:
Stimulus()

#### Add custom stimuli

In [ ]:
import numpy as np

def add_cricket(stimulus_table, bkg_color, bar_color, stim_v=None, skip_duplicates=False):
    stim_id = 10

    if stim_v is None:
        stim_v = f"c{bkg_color}"

    stim_name = 'cricket'
    alias = f"c{bkg_color}_cricket{bkg_color}"

    framerate = 1 / 60.

    stim_info = {
        "nTrials": 5,
        "DirList": [0, 180, 45, 225, 90, 270, 135, 315],
        "tMoveDur_s": 3.0,
        "barDx_um": 75.0,
        "barDy_um": 195.0,
        "vel_umSec": 650.0,
        "bkgColor": (bkg_color, bkg_color, bkg_color),
        "barColor": (bar_color, bar_color, bar_color),
        "durFr_s": 1 / 60.0,
        "nFrPerMarker": 3,
    }

    trialinfo = np.array([0, 180, 45, 225, 90, 270, 135, 315])

    stimulus_table().add_stimulus(
        stim_id=stim_id, stim_v=stim_v, stim_name=stim_name, alias=alias, framerate=framerate,
        isrepeated=1, ntrigger_rep=1, is_colour=0,
        stim_info=stim_info, trialinfo=trialinfo,
        skip_duplicates=skip_duplicates, unique_alias=True)

In [ ]:
for bkg_color in [63, 127, 255]:
    add_cricket(Stimulus, bkg_color=bkg_color, bar_color=0, skip_duplicates=True)

In [ ]:
Stimulus()

#### Presentations

In [ ]:
Presentation().populate()

In [ ]:
Presentation()

### Traces

In [ ]:
Traces().populate()
Traces()

In [ ]:
PreprocessParams().add_default(skip_duplicates=True)
PreprocessParams()

In [ ]:
PreprocessTraces().populate(display_progress=True)
PreprocessTraces()

In [ ]:
PreprocessTraces().plot1(key=PreprocessTraces.fetch(as_dict=True)[0])

In [ ]:
Snippets().populate(display_progress=True)
Snippets()

In [ ]:
Snippets().plot1(key=Snippets.fetch(as_dict=True)[0])

In [ ]:
Averages().populate(display_progress=True)
Averages()

In [ ]:
Averages().plot1(key=Averages.fetch(as_dict=True)[0])

## Recording location

In [ ]:
OpticDisk.populate()
OpticDisk()

In [ ]:
RelativeFieldLocation.populate()
RelativeFieldLocation()

In [ ]:
RelativeFieldLocation().plot()

In [ ]:
RetinalFieldLocation.populate()
RetinalFieldLocation()

In [ ]:
RetinalFieldLocation().plot()